# PyTorch

> Ejemplo adaptado de los tutoriales https://medium.com/apache-mxnet/mxnet-for-pytorch-users-in-10-minutes-a7353863406a, https://mxnet.apache.org/api/python/docs/tutorials/getting-started/to-mxnet/pytorch.html y https://nextjournal.com/gkoehler/pytorch-mnist.

Usamos la biblioteca PyTorch, para lo cual hemos de instalarla previamente (junto con NumPy en caso de que no lo tengamos ya en nuestro sistema). 

Usando `pip`, ejecutamos el siguiente comando:

`pip3 install torch==1.2.0+cpu torchvision==0.4.0+cpu -f https://download.pytorch.org/whl/torch_stable.html` 

Si disponemos de GPU en nuestro ordenador, podemos aprovecharla instalando la versión de MXNet con soporte para GPU:

`pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html`

si estamos utilizando la versión 9.2 de CUDA, o bien

`pip3 install torch===1.2.0 torchvision===0.4.0 -f https://download.pytorch.org/whl/torch_stable.html`

si tenemos instalada la versión 10 de CUDA. Si en nuestro ordenador tenemos otra versión diferente de CUDA, tendremos que seleccionar la versión adecuada de PyTorch tal como se indica en la web: https://pytorch.org/get-started/locally.

Cuando tengamos todo en orden, ya podemos utilizar PyTorch desde Python:

In [1]:
import torch

Empezamos con una comprobación de la versión que hemos instalado:

In [2]:
print(torch.__version__)

1.2.0+cu92


Si no se ha producido ningún error al ejecutar la línea anterior, podemos empezar a trabajar con PyTorch en nuestro ordenador. Para que los experimentos sean reproducibles, podemos establecer la semilla del generador de números pseudoaleatorios...

In [3]:
torch.manual_seed(42)  # Semilla del generador de números aleatorios

Para comprobar si estamos utilizando CUDA desde PyTorch...

In [4]:
print(torch.cuda.is_available())

if torch.cuda.device_count()==1:
    print(torch.cuda.get_device_name(0))
    

True
GeForce GT 755M


c:\python\python363-64\lib\site-packages\torch\cuda\__init__.py:132: UserWarning: 
    Found GPU0 GeForce GT 755M which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


## El conjunto de datos MNIST

Descargamos el conjunto de datos MNIST:

In [5]:
from torchvision import datasets, transforms

trans = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3181,))])

train_data = torch.utils.data.DataLoader(datasets.MNIST('files/', 
    train=True, download=True, transform=trans),
    batch_size=128, shuffle=True, num_workers=4)

test_data = torch.utils.data.DataLoader(datasets.MNIST('files/', 
    train=False, download=True, transform=trans),
    batch_size=128, shuffle=False)


Los valores 0.1307 y 0.3081 utilizados en la normalización corresponden a la media y la desviación estándar del conjunto de datos MNIST.

## Nuestra primera red neuronal en PyTorch

Para nuestros experimentos con PyTorch, utilizaremos el paquete `nn` de PyTorch en el que ya se definen distintos tipos de capas para redes neuronales artificiales:

In [6]:
import torch.nn as pt_nn

Definimos nuestra red neuronal multicapa:

In [7]:
# Definición de la red neuronal

network = pt_nn.Sequential(
    pt_nn.Linear(28*28, 256),
    pt_nn.ReLU(),
    pt_nn.Linear(256, 10))


Especificamos la función de pérdida y el algoritmo de optimización que vamos a utilizar:

In [8]:
loss_function = pt_nn.CrossEntropyLoss()
trainer = torch.optim.SGD(network.parameters(), lr=0.1)

A continuación, entrenamos la red recorriendo una y otra vez el conjunto de entrenamiento:

In [9]:
%%time

import time

for epoch in range(10):
    total_loss = .0
    tic = time.time()
    for X, y in train_data:
        trainer.zero_grad()
        loss = loss_function(network(X.view(-1, 28*28)), y)
        loss.backward()
        trainer.step()
        total_loss += loss.mean()
    print('epoch %d, avg loss %.4f, time %.2f' % (
        epoch, total_loss/len(train_data), time.time()-tic))


epoch 0, avg loss 0.3260, time 5.81
epoch 1, avg loss 0.1514, time 5.43
epoch 2, avg loss 0.1078, time 5.47
epoch 3, avg loss 0.0831, time 5.41
epoch 4, avg loss 0.0677, time 5.43
epoch 5, avg loss 0.0568, time 5.46
epoch 6, avg loss 0.0481, time 5.59
epoch 7, avg loss 0.0414, time 5.56
epoch 8, avg loss 0.0357, time 5.73
epoch 9, avg loss 0.0309, time 6.68
Wall time: 56.6 s


Tras unos segundos, obtenemos una red entrenada capaz de clasificar correctamente más del 90% de los ejemplos del conjunto de entrenamiento de MNIST, algo que comprobamos sobre el conjunto de prueba de MNIST:

In [10]:
test_loss = 0
correct = 0

network.eval()

with torch.no_grad():
    for data, target in test_data:
        output = network(data.view(-1, 28*28))
        test_loss += loss_function(output, target)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_data.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_data.dataset), 100. * correct / len(test_data.dataset)))


Test set: Avg. loss: 0.0005, Accuracy: 9806/10000 (98%)



Si todo ha ido bien, habremos obtenido una red capaz de clasificar correctamente sobre el 98% de los ejemplos del conjunto de prueba de MNIST.